In [11]:
df = pd.read_csv('../PROCESS_CLIM/Val_REF_QCDaily.csv')
print df.shape
df

(2, 6)


,Variable,TMAX,TMIN,RAIN,ESOL,RHUM
0,Vmax,50,35,200,1600,100
1,Vmin,10,0,0,10,35


In [12]:
df = pd.read_csv('../PROCESS_CLIM/Val_REF_QCHour.csv')
print df.shape
df

(2, 6)


,Variable,TMAX,TMIN,RAIN,ESOL,RHUM
0,Vmax,45,35,40,1200,100
1,Vmin,15,5,0,0,40


In [13]:
df = pd.read_csv('../PROCESS_CLIM/Est_Lat.csv')
print df.shape
df

(10, 2)


,ID,Lat
0,21110040,3.074889
1,21110160,3.118472
2,21110180,3.036028
3,21110460,2.939750
4,21115010,3.234472
5,21115020,2.948750
6,21115080,3.222528
7,21115180,3.133167
8,21120040,3.141083
9,21130050,3.243556
